In [1]:
!pip install pennylane

In [2]:
import pennylane as qml
from pennylane.operation import Operation
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:10000]
y_train = y_train[:10000]

x_test = x_test[-2000:]
y_test = y_test[-2000:]

x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x, y

x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)

pca = PCA(n_components=8)
x_train_small = pca.fit_transform(x_train_reshaped)
x_test_small = pca.transform(x_test_reshaped)

In [4]:
class RBSGate(Operation):
    num_params = 1
    num_wires = 2
    par_domain = 'R'

    def __init__(self, theta, wires):
        super().__init__(theta, wires=wires)
        self.theta = theta

    @staticmethod
    def compute_matrix(theta):
        cos = tf.cos(theta)
        sin = tf.sin(theta)
        return tf.convert_to_tensor([
            [1, 0, 0, 0],
            [0, cos, sin, 0],
            [0, -sin, cos, 0],
            [0, 0, 0, 1]
        ], dtype=tf.float64)

    def adjoint(self):
        return RBSGate(-self.parameters[0], wires=self.wires)

    def label(self, decimals=None, base_label=None, **kwargs):
        theta = self.parameters[0]
        return f"RBS({theta:.2f})"

In [5]:
def convert_array(X):
    alphas = tf.zeros(X.shape[:-1] + (X.shape[-1]-1,), dtype=X.dtype)
    X_normd = tf.linalg.l2_normalize(X, axis=-1)
    for i in range(X.shape[-1]-1):
        prod_sin_alphas = tf.reduce_prod(tf.sin(alphas[..., :i]), axis=-1)
        updated_value = tf.acos(X_normd[..., i] / prod_sin_alphas)
        indices = tf.constant([[i]])
        updates = tf.reshape(updated_value, [1])
        alphas = tf.tensor_scatter_nd_update(alphas, indices, updates)
    return alphas

def vector_loader(alphas, wires=None, is_x=True, is_conjugate=False):
    if wires is None:
        wires = list(range(len(alphas) + 1))
    if is_x and not is_conjugate:
        qml.PauliX(wires=wires[0])
    if is_conjugate:
        for i in range(len(wires) - 2, -1, -1):
            qml.apply(RBSGate(-alphas[i], wires=[wires[i], wires[i+1]]))
    else:
        for i in range(len(wires) - 1):
            qml.apply(RBSGate(alphas[i], wires=[wires[i], wires[i+1]]))
    if is_x and is_conjugate:
        qml.PauliX(wires=wires[0])

In [6]:
def nn_circuit(parameters, wires=None):
    if wires is None:
        wires = list(range(len(parameters) // 2))
    n = len(wires)
    for i in range(n-1):
        qml.apply(RBSGate(parameters[i], wires=[wires[n-i-1], wires[n - i -2]]))

In [7]:
dev = qml.device('default.qubit', wires=8)

@qml.qnode(dev, interface='tf', diff_method='backprop')
def quantum_model(inputs, weights):
    inputs = tf.cast(inputs, tf.float64)
    weights = tf.cast(weights, tf.float64)
    vector_loader(convert_array(inputs), wires=range(8))
    nn_circuit(weights, wires=range(8))
    return qml.expval(qml.PauliZ(0))

In [8]:
weights_init = np.random.normal(size=(7,), scale=np.pi/4)
weights = tf.Variable(weights_init, dtype=tf.float64)

class HybridModel(tf.keras.Model):
    def __init__(self, quantum_model):
        super(HybridModel, self).__init__()
        self.quantum_model = quantum_model

    def build(self, input_shape):
        self.quantum_weights = self.add_weight(shape=(7,), initializer='random_normal', trainable=True, dtype=tf.float64)

    def call(self, inputs):
        inputs = tf.cast(inputs, tf.float64)
        outputs = tf.map_fn(lambda x: self.quantum_model(x, self.quantum_weights), inputs, dtype=tf.float64)
        return outputs

model = HybridModel(quantum_model)

In [9]:
def hinge_loss(y_true, y_pred):
    return tf.reduce_mean(tf.maximum(1.0 - y_true * y_pred, 0.0))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

x_train_small_tensor = tf.convert_to_tensor(x_train_small, dtype=tf.float64)
y_train_tensor = tf.convert_to_tensor(2 * y_train - 1, dtype=tf.float64)

@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = hinge_loss(targets, predictions)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.sign(predictions), targets), tf.float64))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, accuracy

In [10]:
@tf.function
def validation_step(inputs, targets):
    predictions = model(inputs)
    loss = hinge_loss(targets, predictions)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.sign(predictions), targets), tf.float64))
    return loss, accuracy

In [12]:
x_test_small_tensor = tf.convert_to_tensor(x_test_small, dtype=tf.float64)
y_test_tensor = tf.convert_to_tensor(2 * y_test - 1, dtype=tf.float64)

epochs = 30
for epoch in range(epochs):
    loss, acc = train_step(x_train_small_tensor, y_train_tensor)
    val_loss, val_acc = validation_step(x_test_small_tensor, y_test_tensor)
    print(f'Epoch {epoch + 1} ----------------------------------- \n Train Loss: {loss.numpy()}, Train Accuracy: {acc.numpy()},  Validation Loss: {val_loss.numpy()}, Validation Accuracy: {val_acc.numpy()}')

Epoch 1 ----------------------------------- 
 Train Loss: 0.9283672288199954, Train Accuracy: 0.552297165200391,  Validation Loss: 0.98103437502747, Validation Accuracy: 0.5134474327628362
Epoch 2 ----------------------------------- 
 Train Loss: 0.9239592918834791, Train Accuracy: 0.555229716520039,  Validation Loss: 0.9791819571772121, Validation Accuracy: 0.5061124694376528
Epoch 3 ----------------------------------- 
 Train Loss: 0.9198439711965595, Train Accuracy: 0.5571847507331378,  Validation Loss: 0.9775080600564474, Validation Accuracy: 0.508557457212714
Epoch 4 ----------------------------------- 
 Train Loss: 0.9160170391716028, Train Accuracy: 0.5645161290322581,  Validation Loss: 0.9759749922128004, Validation Accuracy: 0.511002444987775
Epoch 5 ----------------------------------- 
 Train Loss: 0.9124666848062603, Train Accuracy: 0.5654936461388075,  Validation Loss: 0.9745578765550154, Validation Accuracy: 0.5061124694376528
Epoch 6 ----------------------------------- 
 